In [7]:
import numpy as np
from towbintools.foundation.image_handling import read_tiff_file
import os
import matplotlib.pyplot as plt
import cv2
from skimage.util import img_as_ubyte
import pandas as pd

from skimage import (
    data, restoration, util
)

from joblib import Parallel, delayed
import warnings
from towbintools.foundation.file_handling import get_dir_filemap, add_dir_to_experiment_filemap
warnings.filterwarnings('ignore')

In [8]:
def measure_stack_nuclear_stats(
    raw_stack,
    mask_stack,
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)),
):
    nuclear_stats_df = pd.DataFrame()
    for i, plane in enumerate(raw_stack):
        raw = plane - np.median(plane)
        nuclei_mask = (mask_stack[i] > 0).astype(int)
        nuclei_labels = mask_stack[i]

        unique_labels = np.unique(nuclei_labels)
        # remove label 0
        unique_labels = unique_labels[unique_labels > 0]

        for lbl in unique_labels:
            mask_of_label = (nuclei_labels == lbl).astype(int)
            expanded_label = (
                cv2.morphologyEx(img_as_ubyte(mask_of_label), cv2.MORPH_DILATE, kernel) > 0
            ).astype(int)

            cytoplasm_mask = (expanded_label - mask_of_label - nuclei_mask > 0).astype(int)

            mean_intensity_cytoplasm = np.mean(raw[cytoplasm_mask == 1])
            median_intensity_cytoplasm = np.median(raw[cytoplasm_mask == 1])

            mean_intensity_nucleus = np.mean(raw[nuclei_mask == 1])
            median_intensity_nucleus = np.median(raw[nuclei_mask == 1])

            label_result_dict = {
                    "Plane": i,
                    "Label": lbl,
                    "MeanIntensityCytoplasm": mean_intensity_cytoplasm,
                    "MedianIntensityCytoplasm": median_intensity_cytoplasm,
                    "MeanIntensityNucleus": mean_intensity_nucleus,
                    "MedianIntensityNucleus": median_intensity_nucleus,
                }
            nuclear_stats_df = pd.concat([nuclear_stats_df, pd.DataFrame(label_result_dict, index=[0])])
        
    return nuclear_stats_df

In [9]:
img_dir = '/mnt/towbin.data/shared/spsalmon/20240524_161257_273_LIPSI_40x_397_405_no_crash/raw_ometiff/pad1/'
mask_dir = '/mnt/towbin.data/shared/spsalmon/20240524_161257_273_LIPSI_40x_397_405_no_crash/analysis/ch1_seg_stardist/pad1/'
classification_dir = '/mnt/towbin.data/shared/spsalmon/20240524_161257_273_LIPSI_40x_397_405_no_crash/analysis/nuclei_types/pad1/'

filemap = get_dir_filemap(img_dir)
filemap = add_dir_to_experiment_filemap(filemap, mask_dir, "MaskPath")
filemap = add_dir_to_experiment_filemap(filemap, classification_dir, "ClassificationPath")

output_dir = '/mnt/towbin.data/shared/spsalmon/20240524_161257_273_LIPSI_40x_397_405_no_crash/analysis/nuclei_stats/'
os.makedirs(output_dir, exist_ok=True)
# keep only rows with mask and classification
filemap = filemap[filemap['MaskPath'] != '']
filemap = filemap[filemap['ClassificationPath'] != '']

# for point in filemap['Point'].unique():
#     print(f"Processing point {point}")
#     point_data = filemap[filemap['Point'] == point]

#     for time in point_data['Time'].unique():
#         print(f"Processing time {time}")
#         time_data = point_data[point_data['Time'] == time]
#         raw_path = time_data['ImagePath'].values[0]
#         mask_path = time_data['MaskPath'].values[0]
#         classification_path = time_data['ClassificationPath'].values[0]

#         raw_stack = read_tiff_file(raw_path, channels_to_keep = [1])
#         mask_stack = read_tiff_file(mask_path)
#         classification_df = pd.read_csv(classification_path)

#         nuclei_stats_df = measure_stack_nuclear_stats(raw_stack, mask_stack)
#         nuclei_stats_df = nuclei_stats_df.merge(classification_df, on=['Plane', 'Label'], how='left')

#         output_path = os.path.join(output_dir, os.path.basename(raw_path).replace('.ome.tiff', '.csv'))
#         nuclei_stats_df.to_csv(output_path, index=False)

# parallelize time loop with joblib

def process_time(time, point_data):
    print(f"Processing time {time}")
    time_data = point_data[point_data['Time'] == time]
    raw_path = time_data['ImagePath'].values[0]
    mask_path = time_data['MaskPath'].values[0]
    classification_path = time_data['ClassificationPath'].values[0]

    raw_stack = read_tiff_file(raw_path, channels_to_keep = [1])
    mask_stack = read_tiff_file(mask_path)

    try:
        classification_df = pd.read_csv(classification_path)

        nuclei_stats_df = measure_stack_nuclear_stats(raw_stack, mask_stack)
        nuclei_stats_df = nuclei_stats_df.merge(classification_df, on=['Plane', 'Label'], how='left')

        output_path = os.path.join(output_dir, os.path.basename(raw_path).replace('.ome.tiff', '.csv'))
        nuclei_stats_df.to_csv(output_path, index=False)
    except Exception as e:
        print(f"Error processing time {time}: {e}")

for point in filemap['Point'].unique():
    print(f"Processing point {point}")
    point_data = filemap[filemap['Point'] == point]

    Parallel(n_jobs=1)(
        delayed(process_time)(time, point_data) for time in point_data['Time'].unique()
    )

Processing point 0
Processing time 0
Processing time 6
Processing time 12
Processing time 18
Processing time 24
Processing time 30
Processing time 36
Processing time 42
Processing time 48
Processing time 54
Processing time 60
Processing time 66
Processing time 72
Processing time 78
Processing time 84
Processing time 90
Processing time 96
Processing time 102
Processing time 108
Processing time 114
Processing time 120
Processing time 126
Processing time 132
Processing time 138
Processing time 144
Processing time 150
Processing time 156
Processing time 162
Processing time 168
Processing time 174
Processing time 180
Processing time 186
Processing time 192
Processing time 198
Processing time 204


EmptyDataError: No columns to parse from file